# FAISS DB
- Facebook AI Similarity Search

In [ ]:
# !pip install faiss-cpu
# !pip install faiss-gpu

# 영화 추천 시스템

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
movies = [
    {'id': 0, "title": "인셉션", "description": "꿈속에서 다른 사람의 생각을 조작하는 스토리"},
    {'id': 1, "title": "인터스텔라", "description": "시간과 차원을 넘나드는 우주 탐사 이야기"},
    {'id': 2, "title": "기생충", "description": "두 가족의 계급 갈등을 그린 블랙코미디"},
    {'id': 3, "title": "어벤져스: 엔드게임", "description": "슈퍼히어로들이 우주를 구하기 위해 싸우는 이야기"},
    {'id': 4, "title": "타이타닉", "description": "비극적인 배 사고 속에서 피어난 사랑 이야기"},
    {'id': 5, "title": "라라랜드", "description": "꿈을 쫓는 두 예술가의 로맨스"},
    {'id': 6, "title": "매트릭스", "description": "가상현실과 인공지능의 세상에서 벌어지는 전투"},
    {'id': 7, "title": "쇼생크 탈출", "description": "희망을 잃지 않고 감옥에서 탈출한 한 남자의 이야기"},
    {'id': 8, "title": "해리 포터와 마법사의 돌", "description": "마법 세계에 입학한 소년의 첫 모험"},
    {'id': 9, "title": "반지의 제왕: 반지 원정대", "description": "절대 반지를 파괴하기 위한 여정의 시작"},
    {'id': 10, "title": "다크 나이트", "description": "고담시를 지키기 위한 배트맨과 조커의 대결"},
    {'id': 11, "title": "조커", "description": "한 남자가 광기에 빠져 조커로 변해가는 이야기"},
    {'id': 12, "title": "업", "description": "풍선으로 집을 날리는 노인의 감동적인 모험"},
    {'id': 13, "title": "코코", "description": "죽은 자들의 세계에서 가족의 의미를 깨닫는 이야기"},
    {'id': 14, "title": "겨울왕국", "description": "얼음의 마법을 가진 공주의 자아 찾기 여정"},
    {'id': 15, "title": "주라기 공원", "description": "공룡이 되살아난 테마파크에서 벌어지는 사건"},
    {'id': 16, "title": "킹스맨: 시크릿 에이전트", "description": "신사 스파이들의 스타일리시한 액션"},
    {'id': 17, "title": "미션 임파서블: 고스트 프로토콜", "description": "불가능한 미션을 수행하는 IMF 요원의 활약"},
    {'id': 18, "title": "월-E", "description": "지구 폐허 속 외로운 로봇의 사랑과 모험"},
    {'id': 19, "title": "드라이브", "description": "낮엔 스턴트맨, 밤엔 범죄자의 이중생활"}
]


In [ ]:
descriptions = [movie['description'] for movie in movies]   # 영화 설명 리스트 생성

desc_emb = np.array([model.encode(description) for description in descriptions], dtype='float32')

In [ ]:
dim = desc_emb.shape[1] # 차원 수 (임베딩 벡터의 길이)
idx = faiss.IndexFlatL2(dim)  # L2 거리 기반의 인덱스 생성 (유클리드 거리)  /  chroma db에서 컬렉션을 만들고 데이터를 추가하는 것과 유사한 역할 / 컬렉션은 데이터를 저장하는 것, 인덱스는 검색을 위한 구조체)
idx.add(desc_emb)  # idx에 설명 임베딩 추가 (faiss.IndexFlatL2는 데이터 구조체라고 생각하면 됨) / 데이터를 추가하는 것과 유사한 역할을 함 (faiss는 검색을 위한 구조체, chroma db는 데이터를 저장하는 것)

In [ ]:
# 벡터db에서 검색한다 -> 임베딩된것과 유사한 것을 찾는다는 말. 
# 위에서 임베딩된 것 : descriptions
# idx : collection같은 것. 
# dim : 차원 수 (임베딩 벡터의 길이)

In [ ]:
query_text = "조커"
query_embed = np.array([model.encode(query_text)], dtype='float32')  # 쿼리 텍스트 임베딩 / datatype을 float32로 변환하기 위해 np.array로 감싸줌

In [ ]:
top_n = 5  # 검색할 상위 n개
# faiss는 검색을 위한 구조체이기 때문에 검색을 위한 메소드가 존재
distances, indices = idx.search(query_embed, top_n)  # 쿼리 임베딩과 유사한 상위 5개 검색s
# 결과 : (거리, 인덱스) / 거리 : 쿼리와 가장 가까운 것부터 나옴 / 인덱스 : movies 리스트에서의 인덱스 번호

In [ ]:
print(distances)
print(indices)

[[0.7128261  0.9000239  0.91464114 1.0056337  1.0385551 ]]
[[10 11  2 18 15]]


In [ ]:
for i in range(top_n):
    movie_id = indices[0][i]  # 인덱스 번호로 영화 ID 찾기
    print(f'{i+1}번째 추천: {movies[movie_id]["title"]} (유사도 거리: {distances[0][i]:.2f})')  # 영화 제목과 거리 출력

1번째 추천: 다크 나이트 (유사도 거리: 0.71)
2번째 추천: 조커 (유사도 거리: 0.90)
3번째 추천: 기생충 (유사도 거리: 0.91)
4번째 추천: 월-E (유사도 거리: 1.01)
5번째 추천: 주라기 공원 (유사도 거리: 1.04)


### 사용자 맞춤 뉴스 추천 <실습>

In [ ]:
# 뉴스의 제목을 10, 20 크롤링
# 키워드 입력
# FAISS 활용해 맞춤 뉴스 추천